# Analysis Q-Wave Algorithms

## Setup and Helper Functions

In [1]:
import re
import json

from pathlib import Path

import pandas as pd
import numpy as np
import pingouin as pg

from fau_colors import cmaps, register_fausans_font

import biopsykit as bp
from biopsykit.stats import StatsPipeline

import matplotlib.pyplot as plt
import seaborn as sns

from pepbench.io import load_challenge_results_from_folder
from pepbench.datasets import EmpkinsDataset
from pepbench.data_handling import get_reference_pep, compute_pep_performance_metrics
from pepbench.data_handling.utils import reindex_empkins, reindex_guardian, rename_empkins, rename_guardian

%matplotlib widget
%load_ext autoreload
%autoreload 2

In [2]:
register_fausans_font()
plt.close("all")

palette = sns.color_palette(cmaps.faculties_light)
sns.set_theme(context="notebook", style="ticks", font="sans-serif", palette=palette)

plt.rcParams["figure.figsize"] = (10, 5)
plt.rcParams["pdf.fonttype"] = 42
plt.rcParams["mathtext.default"] = "regular"
plt.rcParams["font.family"] = "sans-serif"
plt.rcParams["font.sans-serif"] = "FAUSans Office"

palette

[(0.37254901960784315, 0.48627450980392156, 0.6392156862745098),
 (0.6666666666666666, 0.7647058823529411, 0.8196078431372549),
 (1.0, 0.8235294117647058, 0.44313725490196076),
 (0.37254901960784315, 0.7725490196078432, 0.9254901960784314),
 (0.5372549019607843, 0.803921568627451, 0.4745098039215686),
 (0.8627450980392157, 0.4117647058823529, 0.48627450980392156)]

In [3]:
root_path = Path("../../")

In [4]:
deploy_type = "local"

config_dict = json.load(root_path.joinpath("config.json").open(encoding="utf-8"))

empkins_base_path = Path(config_dict[deploy_type]["empkins_path"])
guardian_base_path = Path(config_dict[deploy_type]["guardian_path"])
print(empkins_base_path)

/Users/richer/Documents/PhD/Projects/EmpkinS/HealthPsychology_D03/Data/2024_08_PEP_Benchmarking/EmpkinS_Dataset


In [5]:
result_path = root_path.joinpath("results")

In [6]:
dataset_empkins = EmpkinsDataset(empkins_base_path, use_cache=True, only_labeled=True)
dataset_empkins

,participant,condition,phase
0,VP_001,tsst,Prep
1,VP_001,tsst,Pause_1
2,VP_001,tsst,Talk
3,VP_001,tsst,Math
4,VP_001,tsst,Pause_5
...,...,...,...
145,VP_032,ftsst,Prep
146,VP_032,ftsst,Pause_1
147,VP_032,ftsst,Talk
148,VP_032,ftsst,Math


In [26]:
algo_levels = ["q_wave_algorithm", "b_point_algorithm", "outlier_correction_algorithm"]
algo_level_mapping = dict(zip(algo_levels, ["Q-Wave Algorithm", "B-Point Algorithm", "Outlier Correction"]))

In [38]:
results_empkins = load_challenge_results_from_folder(
    result_path.joinpath("empkins_dataset_reference_q_wave"), index_cols_per_sample=["participant", "condition", "phase"]
)

In [ ]:
results_guardian = load_challenge_results_from_folder(
    result_path.joinpath("guardian_dataset_reference_q_wave"), index_cols_per_sample=["participant", "phase"]
)

In [39]:
results_empkins.per_sample

heartbeat_id  \
                                                                                                      estimated   
q_wave_algorithm b_point_algorithm   outlier_correction_algorithm participant condition phase                     
q-wave-reference multiple-conditions autoregression               VP_001      tsst      Prep    0             0   
                                                                                                1             1   
                                                                                                2             2   
                                                                                                3             3   
                                                                                                4             4   
...                                                                                                         ...   
                 third-derivative    none                         VP_032      ftsst     Pause_5 7             7   
                                                                                                8             8   
                                                                                                9             9   
                                                                                                10           10   
                                                                                                11           11   

                                                                                                              \
                                                                                                   reference   
q_wave_algorithm b_point_algorithm   outlier_correction_algorithm participant condition phase                  
q-wave-reference multiple-conditions autoregression               VP_001      tsst      Prep    0          0   
                                                                                                1          1   
                                                                                                2          2   
                                                                                                3          3   
                                                                                                4          4   
...                                                                                                      ...   
                 third-derivative    none                         VP_032      ftsst     Pause_5 7          7   
                                                                                                8          8   
                                                                                                9          9   
                                                                                                10        10   
                                                                                                11        11   

                                                                                                   heartbeat_start_sample  \
                                                                                                                estimated   
q_wave_algorithm b_point_algorithm   outlier_correction_algorithm participant condition phase                               
q-wave-reference multiple-conditions autoregression               VP_001      tsst      Prep    0                     394   
                                                                                                1                     984   
                                                                                                2                    1569   
                                                                                                3                    2134   
                                            

In [40]:
metrics_empkins = compute_pep_performance_metrics(results_empkins.per_sample, num_heartbeats=results_empkins.agg_total)
metrics_empkins

Mean Absolute Error [ms]  \
                                                                                      mean   
q_wave_algorithm b_point_algorithm   outlier_correction_algorithm                            
q-wave-reference straight-line       none                                        14.859426   
                                     autoregression                              15.227649   
                                     linear-interpolation                        15.389102   
                 second-derivative   autoregression                              16.935217   
                                     none                                        17.062323   
                                     linear-interpolation                        17.118264   
                 multiple-conditions linear-interpolation                        21.230078   
                                     autoregression                              21.435405   
                                     none                                        22.556344   
                 third-derivative    none                                        23.319845   
                                     autoregression                              24.735597   
                                     linear-interpolation                        25.053099   

                                                                              \
                                                                         std   
q_wave_algorithm b_point_algorithm   outlier_correction_algorithm              
q-wave-reference straight-line       none                          14.743126   
                                     autoregression                14.727817   
                                     linear-interpolation          14.810659   
                 second-derivative   autoregression                17.106120   
                                     none                          16.819534   
                                     linear-interpolation          17.252294   
                 multiple-conditions linear-interpolation          24.312206   
                                     autoregression                24.362787   
                                     none                          25.208009   
                 third-derivative    none                          18.030187   
                                     autoregression                18.295824   
                                     linear-interpolation          18.388253   

                                                                  Mean Error [ms]  \
                                                                             mean   
q_wave_algorithm b_point_algorithm   outlier_correction_algorithm                   
q-wave-reference straight-line       none                               10.052791   
                                     autoregression                     12.503532   
                                     linear-interpolation               12.842180   
                 second-derivative   autoregression                     -1.197982   
                                     none                               -5.306385   
                                     linear-interpolation               -0.723915   
                 multiple-conditions linear-interpolation              -15.638741   
                                     autoregression                    -16.233442   
                                     none                              -19.784224   
                 third-derivative    none                               19.757422   
                                     autoregression                     23.447948   
                                     linear-interpolation               23.999192   

                                                                              \
                                                                         std   
q_wave_algorithm b

In [42]:
results_empkins.per_sample[("nan_reason", "estimated")].groupby(algo_levels).value_counts().to_frame()

nan_reason
                                                                                                 estimated
q_wave_algorithm b_point_algorithm   outlier_correction_algorithm (nan_reason, estimated)                 
q-wave-reference multiple-conditions autoregression               negative_pep                         103
                                                                  no_monotonic_segment                  38
                                                                  c_point_nan                           14
                                     linear-interpolation         negative_pep                         101
                                                                  no_monotonic_segment                  38
                                                                  c_point_nan                           14
                                     none                         negative_pep                         147
                                                                  no_monotonic_segment                  42
                                                                  c_point_nan                           14
                 second-derivative   autoregression               no_local_minima                      366
                                                                  negative_pep                          40
                                                                  r_peak_or_c_point_nan                  5
                                     linear-interpolation         no_local_minima                      366
                                                                  negative_pep                          40
                                                                  r_peak_or_c_point_nan                  5
                                     none                         no_local_minima                      406
                                                                  r_peak_or_c_point_nan                  5
                 straight-line       autoregression               negative_pep                          40
                                                                  c_point_nan                            5
                                     linear-interpolation         negative_pep                          40
                                                                  c_point_nan                            5
                                     none                         negative_pep                          51
                                                                  c_point_nan                            5
                 third-derivative    autoregression               negative_pep                          48
                                                                  no_c_point                             5
                                     linear-interpolation         negative_pep                          42
                                                                  no_c_point                             5
                                     none                         negative_pep                          46
                                                                  invalid_b_point_search_window         31
                                                                  no_c_point                             5